In [1]:
import pyms

# The following twolines were meant to fix the issue, but didn't :(
import imp
imp.reload(pyms)

import numpy as np
import torch
#TODO: LOWENA, this first cell is just so you can see how this package works, play around with the defocus a bit
# Get crystal
from pyms.utils.torch_utils import (
    amplitude,
    crop_to_bandwidth_limit_torch,
    size_of_bandwidth_limited_array,
    amplitude,
)

from pyms.Probe import (
    make_contrast_transfer_function,
)

crystal = pyms.structure.fromfile(
    "Structures/SrTiO3_CeO2_interface.xyz", atomic_coordinates="cartesian"
)

# A few maniupulations to remove vaccuum at edges and create a psuedo-periodic
# structure
crystal = crystal.resize([0.1, 0.76], axis=0)
from copy import deepcopy

other = deepcopy(crystal).resize([0.017, 0.99], axis=0)
other.reflect([0])
crystal = crystal.concatenate(other, axis=0)


# Quick plot of crystal
# crystal.quickplot(atomscale=1e-5)

# Subslicing of crystal for multislice
subslices = [0.33, 0.66, 1.0]

# Grid size in pixels
gridshape = [1024, 1024]

# Tile structure in multislice for a square grid
tiling = [1, 7]

# Probe accelerating voltage in eV
eV = 3e5

# Objective aperture in mrad
app = 15

# No aperture is also an option
# app = None

# Set up series of thicknesses
thicknesses = np.array([100])

# Set up  series of defocii
df = torch.tensor([200.0])


output = pyms.HRTEM(
    crystal,
    gridshape,
    eV,
    app,
    thicknesses,
    subslices=subslices,
    df=df,
    tiling=tiling,
    nfph=1,
    showProgress='notebook', 
    apply_ctf = False,
)

import matplotlib.pyplot as plt
%matplotlib inline
#plt.imshow(output)


TypeError: HRTEM() got an unexpected keyword argument 'apply_ctf'

In [ ]:
plt.imshow(torch.abs(output).detach().cpu().numpy(), vmax = 10)

In [ ]:
defocus = torch.tensor([100.], requires_grad = True)
bw_limit_size = size_of_bandwidth_limited_array(gridshape)
rsize = np.asarray(crystal.unitcell[:2]) * np.asarray(tiling)
aberrations = []
optimizer = torch.optim.Adam([defocus], lr = 1e-1)
criterion = torch.nn.MSELoss()
n_iters = 2000

path = '/home/Documents/Project/differentiable/df_200_test_data.npy'

ref_data = np.load(path)
ref_data = torch.tensor(ref_data).to('cuda')
loss_curve = []
defocii_curve = []
for iter in range(n_iters):
    ctf = (
            torch.stack(
                [
                    make_contrast_transfer_function(
                        bw_limit_size, rsize, eV, app, df=defocus, aberrations=aberrations
                    )
                ]
            )
        .type(torch.complex128)
        .to('cuda')
    )

    sim_image = amplitude(torch.fft.ifftn(ctf * output, dim=(-2, -1)))
    
    optimizer.zero_grad()
    
    loss = criterion(sim_image, ref_data)*1000000000

    loss.backward()

    #print(defocus.item())

    optimizer.step()

    loss_curve.append(loss.item())
    defocii_curve.append(defocus.item())

In [ ]:
plt.plot(loss_curve)

In [ ]:
plt.imshow(sim_image[0].detach().cpu().numpy())